# Test Cronologia Goal - 420MOOD

Questo notebook testa le funzioni che calcolano e mostrano la cronologia dei goal come implementate nel file `1_Partite.py`.

## Funzioni testate:
- Calcolo del risultato finale
- Estrazione e ordinamento dei goal
- Formattazione per la visualizzazione


In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Aggiungi il path per importare i moduli locali
sys.path.append('../app')

# Importa le funzioni necessarie
from futsal_analysis.utils_time import *
from futsal_analysis.utils_eventi import *

print("✅ Import completati con successo!")


✅ Import completati con successo!


## 1. Creazione Dati di Test

Creiamo un DataFrame di esempio con eventi di una partita fittizia per testare le funzioni.


In [ ]:
# Dati di test - Eventi di una partita fittizia
test_data = {
    'posizione': ['00:01:30', '00:05:45', '00:12:20', '00:15:10', '00:18:30', 
                 '00:20:00', '00:22:15', '00:25:40', '00:28:50', '00:35:00',
                 '00:38:20', '00:40:00'],
    'evento': ['Inizio partita', 'Gol', 'Gol', 'Fine primo tempo', 'Gol',
              'Fine primo tempo', 'Gol', 'Gol', 'Autogol', 'Fine secondo tempo',
              'Gol', 'Fine partita'],
    'squadra': ['', 'Noi', 'Loro', '', 'Noi',
               '', 'Loro', 'Noi', 'Noi', '',
               'Loro', ''],
    'chi': ['', 'Rossi', 'Bianchi', '', 'Verdi',
           '', 'Neri', 'Rossi', 'Gialli', '',
           'Bianchi', ''],
    'dove': [0, 5, 3, 0, 7, 0, 2, 8, 6, 0, 4, 0]
}

df_test = pd.DataFrame(test_data)
print("📊 DataFrame di test creato:")
print(df_test.to_string(index=False))
print(f"\n📈 Shape: {df_test.shape}")


## 2. Calcolo Tempi Effettivi e Reali

Applicamo le funzioni per calcolare i tempi come nel file originale.


In [ ]:
# Data cleaning/normalizzazione (come nel file originale)
df_test.columns = df_test.columns.str.strip().str.lower().str.replace(" ", "_")
df_test = df_test.copy()
df_test['dove'] = pd.to_numeric(df_test.get('dove', None), errors='coerce').fillna(0).astype(int)

# Calcolo dei tempi
df_test['Periodo'] = tag_primo_secondo_tempo(df_test)
df_test['tempoEffettivo'] = calcola_tempo_effettivo(df_test)
df_test['tempoReale'] = calcola_tempo_reale(df_test)

print("⏰ DataFrame con tempi calcolati:")
print(df_test[['posizione', 'evento', 'squadra', 'chi', 'tempoEffettivo', 'tempoReale', 'Periodo']].to_string(index=False))


## 3. Calcolo Risultato Finale

Testiamo il calcolo del risultato finale della partita.


In [ ]:
# Calcolo risultato finale (come nel file originale)
gol_fatti = len(df_test[(df_test['evento'] == 'Gol') & (df_test['squadra'] == 'Noi')])
gol_subiti = len(df_test[(df_test['evento'] == 'Gol') & (df_test['squadra'] == 'Loro')])

print(f"⚽ RISULTATO FINALE:")
print(f"FMP {gol_fatti} – {gol_subiti} Avversario")
print(f"\n📊 Dettaglio goal:")
print(f"• Goal fatti: {gol_fatti}")
print(f"• Goal subiti: {gol_subiti}")
print(f"• Autogol: {len(df_test[df_test['evento'] == 'Autogol'])}")


## 4. Estrazione e Ordinamento Goal

Testiamo l'estrazione e l'ordinamento dei goal per la cronologia.


In [ ]:
# Estrazione goal (come nel file originale)
gol_df = df_test[(df_test['evento'] == 'Gol') | (df_test['evento'] == 'Autogol')].copy()

print("🎯 Goal estratti:")
print(gol_df[['posizione', 'evento', 'squadra', 'chi', 'tempoReale']].to_string(index=False))

if not gol_df.empty:
    gol_df = gol_df.copy()
    gol_df['Minuto'] = gol_df['tempoReale']
    gol_df['Marcatore'] = gol_df['chi'].fillna('').str.title()
    gol_df['Squadra'] = gol_df['squadra']
    
    print("\n📋 Goal con colonne formattate:")
    print(gol_df[['Minuto', 'Marcatore', 'Squadra', 'evento']].to_string(index=False))
    
    # Ordinamento per minuto crescente (MM:SS) usando timedelta
    try:
        gol_df['_minuto_td'] = pd.to_timedelta(gol_df['Minuto'].astype(str))
        gol_df = gol_df.sort_values('_minuto_td').drop(columns=['_minuto_td']).reset_index(drop=True)
        print("\n✅ Ordinamento con timedelta riuscito")
    except Exception as e:
        gol_df = gol_df.sort_values('Minuto').reset_index(drop=True)
        print(f"\n⚠️ Ordinamento con timedelta fallito: {e}")
        print("✅ Ordinamento alternativo applicato")
    
    print("\n📊 Goal ordinati per cronologia:")
    print(gol_df[['Minuto', 'Marcatore', 'Squadra', 'evento']].to_string(index=False))
else:
    print("\n❌ Nessun goal trovato!")


## 5. Formattazione per Visualizzazione

Testiamo la formattazione dei goal per la visualizzazione (simulando l'HTML di Streamlit).


In [2]:
def format_goal_display(row):
    """Formatta un goal per la visualizzazione come nel file originale"""
    minuto_display = f"{row['Minuto']}"
    descrizione = f"{row['Marcatore']}"
    
    if row['Squadra'] == 'Noi':
        return f"{minuto_display}' - {descrizione} (FMP)"
    else:
        return f"{minuto_display}' - 🔴 (Avversario)"

# if not gol_df.empty:
#     print("🏆 CRONOLOGIA GOAL (formattata):")
#     print("=" * 50)
    
#     for _, row in gol_df.iterrows():
#         formatted_goal = format_goal_display(row)
#         print(formatted_goal)
    
#     print("=" * 50)
#     print(f"\n📈 Totale goal: {len(gol_df)}")
#     print(f"📈 Goal FMP: {len(gol_df[gol_df['Squadra'] == 'Noi'])}")
#     print(f"📈 Goal Avversario: {len(gol_df[gol_df['Squadra'] == 'Loro'])}")
#     print(f"📈 Autogol: {len(gol_df[gol_df['evento'] == 'Autogol'])}")
# else:
#     print("❌ Nessun goal da visualizzare")


## 6. Test con Dati Reali (se disponibili)

Proviamo a caricare dati reali da Supabase per testare con partite vere.


In [3]:
# Test con dati reali (opzionale)
try:
    from futsal_analysis.config_supabase import get_supabase_client

    print("🔗 Connessione a Supabase...")
    supabase = get_supabase_client()

    # Carica le partite
    res = supabase.table("partite").select("*").order("data", desc=False).limit(1).execute()
    partite = res.data

    if partite:
        partita_id = partite[0]["id"]
        partita_info = partite[0]

        print(f"✅ Partita trovata: {partita_info['avversario']} - {partita_info['data']}")

        # Carica eventi della partita
        eventi = supabase.table("eventi").select("*").eq("partita_id", partita_id).order("posizione").execute().data
        df_real = pd.DataFrame(eventi)

        if not df_real.empty:
            print(f"✅ {len(df_real)} eventi caricati")

            # Applica le stesse trasformazioni
            df_real.columns = df_real.columns.str.strip().str.lower().str.replace(" ", "_")
            df_real = df_real.copy()
            df_real['dove'] = pd.to_numeric(df_real.get('dove', None), errors='coerce').fillna(0).astype(int)
            df_real['Periodo'] = tag_primo_secondo_tempo(df_real)
            df_real['tempoEffettivo'] = calcola_tempo_effettivo(df_real)
            df_real['tempoReale'] = calcola_tempo_reale(df_real)

            # Calcola risultato
            gol_fatti_real = len(df_real[(df_real['evento'] == 'Gol') & (df_real['squadra'] == 'Noi')])
            gol_subiti_real = len(df_real[(df_real['evento'] == 'Gol') & (df_real['squadra'] == 'Loro')])

            print(f"\n⚽ RISULTATO REALE:")
            print(f"FMP {gol_fatti_real} – {gol_subiti_real} {partita_info['avversario']}")

            # Estrai goal
            gol_df_real = df_real[(df_real['evento'] == 'Gol') | (df_real['evento'] == 'Autogol')].copy()

            # Stampa a video i record relativi ai gol per debug
            print("\n📝 RECORD GREZZI DEI GOAL (DataFrame):")
            if not gol_df_real.empty:
                print(gol_df_real[['evento', 'squadra', 'chi', 'tempoReale', 'posizione']])
            else:
                print("Nessun record goal trovato nel DataFrame.")

            if not gol_df_real.empty:
                gol_df_real['Minuto'] = gol_df_real['tempoReale']
                gol_df_real['Marcatore'] = gol_df_real['chi'].fillna('').str.title()
                gol_df_real['Squadra'] = gol_df_real['squadra']

                # Ordina
                try:
                    gol_df_real['_minuto_td'] = pd.to_timedelta(gol_df_real['Minuto'].astype(str))
                    gol_df_real = gol_df_real.sort_values('_minuto_td').drop(columns=['_minuto_td']).reset_index(drop=True)
                except Exception:
                    gol_df_real = gol_df_real.sort_values('Minuto').reset_index(drop=True)

                print(f"\n🏆 CRONOLOGIA GOAL REALE:")
                print("=" * 50)

                for idx, row in gol_df_real.iterrows():
                    print(f"DEBUG idx={idx} | evento={row.get('evento')} | squadra={row.get('squadra')} | chi={row.get('chi')} | tempoReale={row.get('tempoReale')} | posizione={row.get('posizione')}")
                    formatted_goal = format_goal_display(row)
                    print(formatted_goal)

                print("=" * 50)
            else:
                print("\n❌ Nessun goal trovato nella partita reale")
        else:
            print("❌ Nessun evento trovato per la partita")
    else:
        print("❌ Nessuna partita trovata")

except Exception as e:
    print(f"⚠️ Errore nel caricamento dati reali: {e}")
    print("ℹ️ Continuo con i dati di test")


🔗 Connessione a Supabase...
✅ Partita trovata: Futsal Bissuola - 2025-09-27
✅ 292 eventi caricati

⚽ RISULTATO REALE:
FMP 3 – 5 Futsal Bissuola

📝 RECORD GREZZI DEI GOAL (DataFrame):
    evento squadra   chi tempoReale    posizione
18     Gol    Loro            04:30  0:13:40.967
36     Gol    Loro            10:56  0:20:06.834
116    Gol    Loro            34:54  0:44:05.500
136    Gol     Noi            43:35  0:52:46.100
220    Gol    Loro            21:33  1:28:15.300
262    Gol     Noi  igor      31:10  1:37:51.600
283    Gol     Noi  azza      35:48  1:42:29.700
287    Gol    Loro            37:43  1:44:25.334

🏆 CRONOLOGIA GOAL REALE:
DEBUG idx=0 | evento=Gol | squadra=Loro | chi= | tempoReale=04:30 | posizione=0:13:40.967
04:30' - 🔴 (Avversario)
DEBUG idx=1 | evento=Gol | squadra=Loro | chi= | tempoReale=10:56 | posizione=0:20:06.834
10:56' - 🔴 (Avversario)
DEBUG idx=2 | evento=Gol | squadra=Loro | chi= | tempoReale=21:33 | posizione=1:28:15.300
21:33' - 🔴 (Avversario)
DEBUG id

In [4]:
## 7. Analisi del Problema con i Tempi Effettivi

Analizziamo i dati per capire perché i tempi effettivi vanno oltre i 40 minuti.


SyntaxError: invalid syntax (754476488.py, line 3)

In [5]:
# Analizziamo i dati problematici
print("🔍 ANALISI DEI DATI PROBLEMATICI:")
print("=" * 60)

# Mostra i goal con i tempi problematici
gol_problematici = gol_df_real[gol_df_real['tempoReale'].str.contains('3[4-9]:|4[0-9]:', regex=True)]
print("Goal con tempi impossibili (> 20 minuti nel primo tempo):")
print(gol_problematici[['evento', 'squadra', 'chi', 'tempoReale', 'posizione']].to_string(index=False))

print("\n" + "=" * 60)
print("📊 ANALISI DELLE COLONNE TEMPO:")

# Analizza la struttura dei tempi
print("\n1. Colonna 'posizione' (tempo reale cronologico):")
print("   Esempi:", df_real['posizione'].head(10).tolist())

print("\n2. Colonna 'tempoReale' (tempo di gioco):")
print("   Esempi:", df_real['tempoReale'].head(10).tolist())

print("\n3. Colonna 'tempoEffettivo' (tempo calcolato):")
print("   Esempi:", df_real['tempoEffettivo'].head(10).tolist())

# Trova gli indici dei momenti chiave
idx_fine_primo = df_real[df_real['evento'] == 'Fine primo tempo'].index[0] if 'Fine primo tempo' in df_real['evento'].values else None
idx_fine_partita = df_real[df_real['evento'] == 'Fine partita'].index[0] if 'Fine partita' in df_real['evento'].values else None

print(f"\n4. Indici chiave:")
print(f"   Fine primo tempo: {idx_fine_primo}")
print(f"   Fine partita: {idx_fine_partita}")

if idx_fine_primo:
    print(f"\n5. Tempi al momento 'Fine primo tempo':")
    print(f"   posizione: {df_real.loc[idx_fine_primo, 'posizione']}")
    print(f"   tempoReale: {df_real.loc[idx_fine_primo, 'tempoReale']}")
    print(f"   tempoEffettivo: {df_real.loc[idx_fine_primo, 'tempoEffettivo']}")


🔍 ANALISI DEI DATI PROBLEMATICI:
Goal con tempi impossibili (> 20 minuti nel primo tempo):
evento squadra  chi tempoReale   posizione
   Gol    Loro           34:54 0:44:05.500
   Gol     Noi azza      35:48 1:42:29.700
   Gol    Loro           37:43 1:44:25.334
   Gol     Noi           43:35 0:52:46.100

📊 ANALISI DELLE COLONNE TEMPO:

1. Colonna 'posizione' (tempo reale cronologico):
   Esempi: ['0:09:10.667', '0:09:27.934', '0:09:50.567', '0:09:51.533', '0:10:07.500', '0:10:27.034', '0:10:27.300', '0:10:38.467', '0:11:07.734', '0:11:24.400']

2. Colonna 'tempoReale' (tempo di gioco):
   Esempi: ['00:00', '00:17', '00:39', '00:40', '00:56', '01:16', '01:16', '01:27', '01:57', '02:13']

3. Colonna 'tempoEffettivo' (tempo calcolato):
   Esempi: ['00:00', '00:08', '00:18', '00:18', '00:25', '00:34', '00:34', '00:39', '00:52', '01:00']

4. Indici chiave:
   Fine primo tempo: 141
   Fine partita: 291

5. Tempi al momento 'Fine primo tempo':
   posizione: 0:53:57.534
   tempoReale: 44:46
 

In [7]:
# Analizziamo la funzione calcola_tempo_effettivo solo per gli eventi Gol
print("🔧 ANALISI DELLA FUNZIONE calcola_tempo_effettivo (solo eventi Gol):")
print("=" * 60)

def debug_calcola_tempo_effettivo_gol(df):
    """Versione debug della funzione calcola_tempo_effettivo SOLO per eventi Gol"""
    df_debug = df.copy()
    df_debug['Posizione_sec'] = df_debug['posizione'].apply(to_seconds)
    
    idx_start = 0
    idx_fine_primo = df_debug[df_debug['evento'] == 'Fine primo tempo'].index[0]
    idx_start_secondo = idx_fine_primo + 1
    idx_fine_partita = df_debug[df_debug['evento'] == 'Fine partita'].index[0]

    t_start = df_debug.loc[idx_start, 'Posizione_sec']
    t_fine_primo = df_debug.loc[idx_fine_primo, 'Posizione_sec']
    t_start_secondo = df_debug.loc[idx_start_secondo, 'Posizione_sec']
    t_fine_partita = df_debug.loc[idx_fine_partita, 'Posizione_sec']
    
    print(f"Tempi di riferimento (in secondi):")
    print(f"  t_start: {t_start} ({format_mmss(t_start)})")
    print(f"  t_fine_primo: {t_fine_primo} ({format_mmss(t_fine_primo)})")
    print(f"  t_start_secondo: {t_start_secondo} ({format_mmss(t_start_secondo)})")
    print(f"  t_fine_partita: {t_fine_partita} ({format_mmss(t_fine_partita)})")
    
    print(f"\nDurata primo tempo: {t_fine_primo - t_start} secondi ({format_mmss(t_fine_primo - t_start)})")
    print(f"Durata secondo tempo: {t_fine_partita - t_start_secondo} secondi ({format_mmss(t_fine_partita - t_start_secondo)})")
    
    # Filtra solo i goal (e autogol se vuoi)
    df_gol = df_debug[df_debug['evento'].str.lower().isin(['gol', 'autogol'])].copy()
    print(f"\nCalcolo per tutti gli eventi Gol ({len(df_gol)}):")
    print("-" * 80)
    
    for i, (idx, row) in enumerate(df_gol.iterrows()):
        t = row['Posizione_sec']
        evento = row['evento']
        squadra = row['squadra']
        
        print(f"\nGol {i+1} (idx={idx}): {evento} - {squadra}")
        print(f"  posizione: {row['posizione']} -> {t} secondi ({format_mmss(t)})")
        
        if idx <= idx_fine_primo:
            perc = (t - t_start) / (t_fine_primo - t_start)
            eff = perc * 20 * 60
            print(f"  PRIMO TEMPO: perc = ({t} - {t_start}) / ({t_fine_primo} - {t_start}) = {perc:.3f}")
            print(f"  eff = {perc:.3f} * 20 * 60 = {eff:.1f} secondi ({format_mmss(eff)})")
        elif idx >= idx_start_secondo:
            perc = (t - t_start_secondo) / (t_fine_partita - t_start_secondo)
            eff = 20 * 60 + perc * 20 * 60
            print(f"  SECONDO TEMPO: perc = ({t} - {t_start_secondo}) / ({t_fine_partita} - {t_start_secondo}) = {perc:.3f}")
            print(f"  eff = 20*60 + {perc:.3f} * 20 * 60 = {eff:.1f} secondi ({format_mmss(eff)})")
        else:
            print(f"  INTERVALLO: eff = NaN")

# Esegui il debug solo per i goal
debug_calcola_tempo_effettivo_gol(df_real)


🔧 ANALISI DELLA FUNZIONE calcola_tempo_effettivo (solo eventi Gol):
Tempi di riferimento (in secondi):
  t_start: 550.667 (09:11)
  t_fine_primo: 3237.534 (53:58)
  t_start_secondo: 4001.567 (66:42)
  t_fine_partita: 6324.3 (105:24)

Durata primo tempo: 2686.867 secondi (44:47)
Durata secondo tempo: 2322.733 secondi (38:43)

Calcolo per tutti gli eventi Gol (8):
--------------------------------------------------------------------------------

Gol 1 (idx=18): Gol - Loro
  posizione: 0:13:40.967 -> 820.967 secondi (13:41)
  PRIMO TEMPO: perc = (820.967 - 550.667) / (3237.534 - 550.667) = 0.101
  eff = 0.101 * 20 * 60 = 120.7 secondi (02:01)

Gol 2 (idx=36): Gol - Loro
  posizione: 0:20:06.834 -> 1206.834 secondi (20:07)
  PRIMO TEMPO: perc = (1206.834 - 550.667) / (3237.534 - 550.667) = 0.244
  eff = 0.244 * 20 * 60 = 293.1 secondi (04:53)

Gol 3 (idx=116): Gol - Loro
  posizione: 0:44:05.500 -> 2645.5 secondi (44:06)
  PRIMO TEMPO: perc = (2645.5 - 550.667) / (3237.534 - 550.667) = 0.78